In [5]:
using ExpandLGF
using OffsetArrays
include("../test/test_utils.jl");

Time the evaluation of near-field LGF entries for dimension-split stencils using a combination of quadratue and series expansion. 

Times are reported for the generation of expansions via symbolic computation, the evalution of near-field values using combined quadrature and expansion, and the evaluation of far-field values using a separate series expansion.

In [2]:
###
# Parameters
###
N = 64            # Time evaluation on [0, N]^3
M = 128           # Test residual on [0, M]^3
near_terms = 10   # terms in near-field expansions
far_terms = 8     # terms in far-field expansion
rtol = 0          # relative tolerance on evaluations
atol = 1e-15      # absolute tolerance on evaluations

1.0e-15

In [3]:

function get_timings(name, stencil)

    ###
    # Time the generation of all the expansion objects
    ###
    expansion_time = @elapsed begin
        ELGF = ExpandLGF.EvaluateLGF(stencil, near_terms, far_terms, rtol, atol)
        nmax = ExpandLGF.near_cutoff(ELGF)
        NF = ExpandLGF.near_field(ELGF)
        FF = ExpandLGF.far_field(ELGF)
    end

    ###
    # Time the near and far field evaluations
    ###
    lgf = OffsetArray(zeros(Float64, (N, N, N)), 0:N-1, 0:N-1, 0:N-1)
    evaluation_indices = sorted_indices_3D(N-1)
    near_field_indices = collect(n for n in evaluation_indices if sum(n.^2) < nmax^2)
    far_field_indices = collect(n for n in evaluation_indices if sum(n.^2) >= nmax^2)

    println("Evaluating near-field for $(name)")
    println("$(stencil)")
    println("Total near-field evaluations: $(length(near_field_indices)) for points with |n| < $(nmax)")
    println("Total far-field evaluations: $(length(far_field_indices)) for points with |n| ≥ $(nmax)")

    near_field_time = @elapsed for index in near_field_indices
        lgf[index...] = NF(index)
    end

    far_field_time = @elapsed for index in far_field_indices
        lgf[index...] = FF(index)
    end

    println("$(expansion_time)s for expansion setup")
    println("$(near_field_time)s for near field")
    println("$(far_field_time)s for far field")

    ###
    # Build a larger block of size M^3
    ###
    filled_block = generate_full_test_block(stencil, lgf)
    test_block = OffsetArray(zeros(Float64, (M + 1, M + 1, M + 1)), 0:M, 0:M, 0:M)
    test_block[0:N-1, 0:N-1, 0:N-1] = filled_block[0:N-1, 0:N-1, 0:N-1]

    FF_flups = ExpandLGF.FarField(stencil, 4)
    for n in CartesianIndices(test_block)
        n = Tuple(n)
        if maximum(n) >= N
            test_block[n...] = FF_flups([n...])
        end
    end

    ###
    # Evaluate the max residual on the larger block
    ###
    residual = apply_stencil(stencil, test_block)
    handle_origin!(residual)
    max_residual, index = findmax(abs, residual)
    @printf("Max residual in test block of size [0, %d]^3: %1.3e", M, max_residual)
    print(" at index $(Tuple(index))\n\n")

end

get_timings (generic function with 1 method)

In [4]:
###
# Print timing results for each stencil
###
# Includes an extra LGF2 to avoid timing precompilation in later runs
names = "LGF" .* string.([2, 2, 4, 6, 8]) 
stencils = ExpandLGF.StandardDifference3D.([2, 2, 4, 6, 8])

for (name, stencil) in zip(names, stencils)
    get_timings(name, stencil)
end

Evaluating near-field for LGF2
3D SplitStencil with coefficients Rational{Int64}[1//1]
Total near-field evaluations: 779 for points with |n| < 19
Total far-field evaluations: 44981 for points with |n| ≥ 19
12.9054315s for expansion setup
8.839778959s for near field
1.585461875s for far field
Max residual in test block of size [0, 128]^3: 2.255e-15 at index (19, 2, 2)

Evaluating near-field for LGF2
3D SplitStencil with coefficients Rational{Int64}[1//1]
Total near-field evaluations: 779 for points with |n| < 19
Total far-field evaluations: 44981 for points with |n| ≥ 19
6.720603417s for expansion setup
5.757737083s for near field
0.3258195s for far field
Max residual in test block of size [0, 128]^3: 2.255e-15 at index (19, 2, 2)

Evaluating near-field for LGF4
3D SplitStencil with coefficients Rational{Int64}[4//3, -1//12]
Total near-field evaluations: 672 for points with |n| < 18
Total far-field evaluations: 45088 for points with |n| ≥ 18
6.714544042s for expansion setup
7.62807825s 